In [ ]:
def evaluate_reports_node(self, state:BasicState):
    # Extract report blocks using regex
    generated_reports = []
    need_regenerate_flags = []
    pattern = r"```report\s+(.*?)```"
    evaluated_result = state["evaluated_result"]
    report_blocks = re.findall(pattern, evaluated_result, re.DOTALL)
    
    for report in report_blocks:
        generated_reports.append(report)
        # Extract the Need_regenerate flag
        need_regenerate_match = re.search(r"Need_regenerate:\s*(True|False)", report)
        if need_regenerate_match:
            need_regenerate = need_regenerate_match.group(1) == "True"
            need_regenerate_flags.append(need_regenerate)
        else:
            # Default to False if not found
            need_regenerate_flags.append(False)
    
    state["generated_reports"] = generated_reports
    state["need_regenerate_flags"] = need_regenerate_flags
    state["any_need_regenerate"] = any(need_regenerate_flags)
    
    # Decide next step based on evaluation results
    if state["any_need_regenerate"]:
        return "errors_found"
    else:
        return "all_passed"

def regenerate_problem_blocks_node(self, state:BasicState):
    """
    Regenerate only the code blocks that have errors.
    """
    # Only regenerate code blocks that need it
    blocks_to_regenerate = []
    for i, (code, report, need_regenerate) in enumerate(zip(
            state["generated_codes"], 
            state["generated_reports"], 
            state["need_regenerate_flags"])):
        if need_regenerate:
            blocks_to_regenerate.append((i, code, report))
    
    # Create a prompt to regenerate specific blocks
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that helps the user to fix errors in deepproblog code."),
        ("human", self.regenerate_prompt)
    ])
    
    code_with_report_list = []
    for i, code, report in blocks_to_regenerate:
        code_with_report_list.append(f"### The {self._ordinal(i)} Code Block:")
        code_with_report_list.append(code)
        code_with_report_list.append(f"It's corresponding analysis:")
        code_with_report_list.append(report)
        code_with_report_list.append("\n")
    
    # Prepare input for the regeneration prompt
    input={
        "rule_set": state["generated_result"],
        "blocks_with_analysis": "\n".join(code_with_report_list),
    }
    
    # Run the regeneration
    chain = prompt | self.model | StrOutputParser()
    result = chain.invoke(input=input, config=self.config)
    
    # Extract the regenerated code blocks
    pattern = r"```(?:prolog|[a-z]*)?\n(.*?)```"
    new_blocks = re.findall(pattern, result, re.DOTALL)
    
    # Replace only the blocks that needed regeneration
    updated_generated_codes = state["generated_codes"].copy()
    for (i, _, _), new_block in zip(blocks_to_regenerate, new_blocks):
        print(f"Regenerated block {i}:")
        print("% === % Old Code % === %")
        print(state["generated_codes"][i])
        print("% === % New Code % === %")
        print(new_block)
        # Update the code with the regenerated version
        updated_generated_codes[i] = "% === % LLM Regenerated Logic Codes % === %\n" + new_block + "\n% === % ========================= % === %\n"
    
    with open(os.path.join(cf.CURRENT_PATH, "history/new_blocks.txt"), "w") as f:
        f.write("\n".join(new_blocks))
    
    return {
        "regenerated_codes": updated_generated_codes
    }

def reconstruct_full_rule_node(self, state:BasicState):
    """
    Reconstruct the full rule with updated code blocks after regeneration.
    """
    # Use the regenerated code blocks to create the updated rule
    updated_result = self._replace_placeholder(state["prompt_rule_template"], state["regenerated_codes"], self.langda_placeholder)
    
    # Write the updated rule to the output file
    with open(cf.GENERATED_RULE_PATH, "w") as f:
        f.write(updated_result)
    
    return {
        "generated_codes": state["regenerated_codes"],
        "generated_result": updated_result
    }

def call_main_workflow(self):
    """
    Build and execute the main workflow graph according to the diagram structure:
    
    Code Generation -> Test Generated Code -> Evaluate Reports -> [All Passed] -> End - Code Complete
                                                               -> [Errors Found] -> Regenerate Problem Blocks -> Reconstruct Full Rule -> Test Generated Code
    """
    main_workflow = StateGraph(BasicState)
    
    # Define all nodes for the main workflow
    main_workflow.add_node("test_generated_code", self.test_node)
    main_workflow.add_node("evaluate_reports", self.evaluate_reports_node)
    main_workflow.add_node("regenerate_problem_blocks", self.regenerate_problem_blocks_node)
    main_workflow.add_node("reconstruct_full_rule", self.reconstruct_full_rule_node)
    
    # Set the entry point to the test node
    main_workflow.set_entry_point("test_generated_code")
    
    # Define the main workflow edges
    main_workflow.add_edge("test_generated_code", "evaluate_reports")
    
    # Add conditional edges from the evaluation node
    main_workflow.add_conditional_edges(
        "evaluate_reports",
        lambda state: "all_passed" if not state["any_need_regenerate"] else "errors_found",
        {
            "all_passed": END,  # End workflow if all tests pass
            "errors_found": "regenerate_problem_blocks"  # Go to regeneration if errors are found
        }
    )
    
    # Complete the regeneration loop
    main_workflow.add_edge("regenerate_problem_blocks", "reconstruct_full_rule")
    main_workflow.add_edge("reconstruct_full_rule", "test_generated_code")  # Loop back to testing
    
    # Compile the main workflow
    compiled_main_workflow = main_workflow.compile()
    
    # Generate a mermaid diagram for visualization
    self._draw_mermaid_png(main_workflow, "main_workflow")
    
    return compiled_main_workflow

def call_gen_workflow(self):
    """
    Run the complete workflow by first executing the generation subgraph
    and then the main workflow.
    """
    # First, run the generation subgraph
    gen_subgraph = self.call_gen_subgraph()
    self.state = gen_subgraph.invoke(self.state, config=self.config)
    
    # Then, run the main workflow
    main_workflow = self.call_main_workflow()
    self.state = main_workflow.invoke(self.state, config=self.config)
    
    # Output the final result
    print("===============generated_result:===============\n")
    print(self.state["generated_result"])

In [3]:
import json

str = """
{
    "Report": "correct, it's suitable for the current code",
    "Need_regenerate": "False"
}
"""
obj = json.loads(str)
print(obj["Need_regenerate"])
print(type(obj))

False
<class 'dict'>


In [9]:
def _replace_placeholder(template, replacement_list:list, placeholder="{{LANGDA}}"):
    segments = template.split(placeholder)
    # segments[0] + (N times [replacement or placeholder] + segments[1..])
    result = segments[0]
    # 遍历每个段后面应该插入的内容
    for i, seg in enumerate(segments[1:]):
        if i < len(replacement_list) and replacement_list[i] is not None:
            result += replacement_list[i]
        else:
            result += placeholder
        result += seg
    return result

rlist = ["ass",None,"hole"]
template = """
first
{{LANGDA}}
second
{{LANGDA}}
third
{{LANGDA}}
what?
"""
res = _replace_placeholder(template, rlist)
print(type(res))



<class 'str'>


In [ ]:
state = {}
def _decide_next(state):
    print("processing _decide_next...")

    to_end = True
    for report in state["generated_reports"]:
        print(report["Need_regenerate"])
        if report["Need_regenerate"] == True:
            to_end = False
    if to_end or state["iter_count"] >= 3:
        state["status"] = TaskStatus.COMPLETED
        return "summary_node"
    else:
        return "regenerate_node"